<a href="https://colab.research.google.com/github/siva-Parvathi-M/My-_Projects/blob/main/Data_Workflow_Automation_Using_Python_and_Gen_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount your Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd

# Define the file path
file_path = '/content/drive/MyDrive/Colab Notebooks/Source Data/online_retail_II.xlsx'

# Load the Excel file (it has 2 sheets: 'Year 2009-2010' and 'Year 2010-2011')
# We'll load both sheets into separate DataFrames
df_2009 = pd.read_excel(file_path, sheet_name='Year 2009-2010')

# Combine both years into one DataFrame
df = pd.concat([df_2009], ignore_index=True)

# Display the first 5 rows
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [14]:
# Show all column names in the loaded DataFrame
print(df.columns.tolist())

['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


In [16]:
df = df.drop_duplicates()
df = df.dropna(subset=['Customer ID'])
df = df[~df['Invoice'].astype(str).str.startswith('C')]
df = df[(df['Quantity'] > 0) & (df['Price'] > 0)]
df.reset_index(drop=True, inplace=True)
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


**Use GenAI (like GPT) for Report Generation**

In [28]:
import json

# Show raw JSON response to debug
print(json.dumps(result, indent=2))

{
  "id": "nvxcfQD-4yUbBN-9474a182ba7b343f-PDX",
  "object": "text.completion",
  "created": 1748507439,
  "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
  "choices": [
    {
      "index": 0,
      "text": "\n---\n\nTitle: Customer Trends and Key Observations from the Top 5 Countries in the 2009-2010 Online Retail Dataset\n\nIntroduction:\n\nThe 2009-2010 Online Retail dataset provides valuable insights into consumer behavior from five top countries: United Kingdom, Denmark, Netherlands, EIRE, and France. This report summarizes the customer trends and key observations from the analysis of the quantity purchased from these countries.\n\nKey Observations:\n\n1. United Kingdom leads in purchases:\n   - The United Kingdom accounts for 73.5% of the total quantity purchased, indicating a strong preference for online shopping in this country.\n   - The high volume of purchases suggests a well-established e-commerce market and a tech-savvy population.\n\n2. Denmark and Netherlands follow cl

In [30]:
import requests
import json

TOGETHER_API_KEY = "73694d96412df7fd1358d0605b15efcf7f045e3f2771d68d0204669d97a73a94"  # 🔁 Replace this

summary_data = df.groupby("Country")["Quantity"].sum().sort_values(ascending=False).head(5)
prompt = f"""
You are a data analyst. Analyze the following top 5 countries by quantity purchased from the 2009-2010 Online Retail dataset:

{summary_data.to_string()}

Generate a professional report summarizing customer trends and key observations.
"""

url = "https://api.together.xyz/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {TOGETHER_API_KEY}",
    "Content-Type": "application/json"
}

data = {
    "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",  # ✅ valid chat model
    "messages": [
        {"role": "system", "content": "You are a helpful data analyst."},
        {"role": "user", "content": prompt}
    ],
    "max_tokens": 300,
    "temperature": 0.7
}

response = requests.post(url, headers=headers, json=data)

# Print full response
result = response.json()
print(json.dumps(result, indent=2))

# Now extract the result safely
if 'choices' in result:
    print("\nGenerated Report:\n")
    print(result['choices'][0]['message']['content'])
else:
    print("❌ No choices found in the response. Check the above output.")

{
  "id": "nvxdJRS-4yUbBN-9474a48cee2c07e3",
  "object": "chat.completion",
  "created": 1748507563,
  "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
  "prompt": [],
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": " Report Title: Top 5 Countries by Quantity Purchased from the 2009-2010 Online Retail Dataset\n\nIntroduction:\nThis report provides an analysis of the top 5 countries by quantity purchased from the 2",
        "tool_calls": []
      },
      "logprobs": null,
      "finish_reason": "stop",
      "seed": null
    }
  ],
  "usage": {
    "prompt_tokens": 126,
    "total_tokens": 181,
    "completion_tokens": 55
  }
}

Generated Report:

 Report Title: Top 5 Countries by Quantity Purchased from the 2009-2010 Online Retail Dataset

Introduction:
This report provides an analysis of the top 5 countries by quantity purchased from the 2


**Automate Multiple Insight Reports with Python & GenAI**

In [31]:
def generate_report(prompt_text):
    url = "https://api.together.xyz/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "mistralai/Mixtral-8x7B-Instruct-v0.1",
        "messages": [
            {"role": "system", "content": "You are a helpful data analyst."},
            {"role": "user", "content": prompt_text}
        ],
        "max_tokens": 300,
        "temperature": 0.7
    }

    response = requests.post(url, headers=headers, json=data)
    result = response.json()

    if 'choices' in result:
        return result['choices'][0]['message']['content']
    else:
        return "Error: No response from model."

In [32]:
# 1. Top Selling Products
top_products = df.groupby("Description")["Quantity"].sum().sort_values(ascending=False).head(5)
prompt_1 = f"""
Analyze the following top 5 selling products by quantity sold:

{top_products.to_string()}

Generate a summary of why these items might be popular and their business value.
"""
print("📦 Top Products Report:\n", generate_report(prompt_1))


# 2. Monthly Sales Trend
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["Month"] = df["InvoiceDate"].dt.to_period("M")
monthly_sales = df.groupby("Month")["Quantity"].sum().sort_values(ascending=False).head(6)
prompt_2 = f"""
Based on the monthly sales trend below (by quantity):

{monthly_sales.to_string()}

Summarize peak months and explain any possible reasons or patterns.
"""
print("\n📈 Monthly Sales Report:\n", generate_report(prompt_2))


# 3. Country-wise Total Sales
country_sales = df.groupby("Country")["Quantity"].sum().sort_values(ascending=False).head(5)
prompt_3 = f"""
Analyze the following top 5 countries by quantity of purchases:

{country_sales.to_string()}

Summarize the key customer markets and possible sales strategies.
"""
print("\n🌍 Country Sales Report:\n", generate_report(prompt_3))

📦 Top Products Report:
  Based on the given data, the top 5 selling products by quantity sold are:

1. WHITE HANGING HEART T-LIGHT HOLDER - 56,814 units
2. WORLD WAR 2 GLIDERS ASSTD DESIGNS - 54,610 units
3. BROCADE RING PURSE - 48,166 units
4. PACK OF 72 RETRO SPOT CAKE CASES - 45,129 units
5. ASSORTED COLOUR BIRD ORNAMENT - 44,431 units

These items are likely popular due to a combination of factors such as attractive design, competitive pricing, and strong demand in the market. To understand their business value, let's examine each item more closely:

1. WHITE HANGING HEART T-LIGHT HOLDER - With over 56,000 units sold, this product is a clear bestseller. Its popularity can be attributed to its stylish design, which likely appeals to a wide range of customers. Its high sales volume generates significant revenue and contributes to a positive brand image.
2. WORLD WAR 2 GLIDERS ASSTD DESIGNS - This historical collectible has found a strong market with 54,610

📈 Monthly Sales Report:
  

In [36]:
!pip install fpdf

In [ ]:
from fpdf import FPDF

# Initialize PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)

# Title
pdf.set_font("Arial", 'B', 16)
pdf.cell(200, 10, txt="Customer Purchase Report (2009–2010)", ln=True, align="C")
pdf.ln(10)

# Function to add sections to the PDF
def add_section(title, content):
    pdf.set_font("Arial", 'B', 14)
    pdf.multi_cell(0, 10, title)
    pdf.set_font("Arial", '', 12)
    pdf.multi_cell(0, 10, content)
    pdf.ln(5)

# Add all GenAI-generated reports
add_section("📦 Top Products Report", generate_report(prompt_1))
add_section("📈 Monthly Sales Trend", generate_report(prompt_2))
add_section("🌍 Country Sales Report", generate_report(prompt_3))

# Save the PDF
pdf.output("GenAI_Report.pdf")